In [ ]:
from enum import Enum

class Relationship(Enum):
    """Enum for relationship types."""
    FRIEND = "friend"
    FAMILY = "family"
    COLLEAGUE = "colleague"
    ACQUAINTANCE = "acquaintance"
    PARTNER = "partner"
    OTHER = "other"
    PARENT = "parent"
    CHILD = "child"
    
class Person:
    """Class representing a person with a name and a relationship."""
    
    def __init__(self, name: str, relationship: Relationship):
        self.name = name
        self.relationship = relationship
    
    def __repr__(self):
        return f"Person(name={self.name}, relationship={self.relationship})"
    
class Relationships:
    """Class to manage a collection of Person objects."""
    
    def __init__(self):
        self.relations = []
        
    def add_parent_and_child(self, parent: Person, child: Person):
        """Add a parent-child relationship."""
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.CHILD, parent))
        
class Research:
    """Class to manage research data."""
    
    def __init__(self, relationships):
        # relationships is a storage implementation and it should be injected as a dependency
        # and it should provide a utility method to get the relations and the search
        relations = relationships.relations
        
        for relation in relations:
            if relation[0].name == 'John' and relation[1] == Relationship.PARENT:
                print(f"John is a parent of {relation[2].name}")
    # this class is accessing the relationships directly - "Relationships" is a low level module and "Research" is a high level module
    # this violates the dependency inversion principle
                
parent = Person("John", Relationship.PARENT)
child = Person("Jane", Relationship.CHILD)
child2 = Person("Jack", Relationship.CHILD)

relationships = Relationships()
relationships.add_parent_and_child(parent, child)
relationships.add_parent_and_child(parent, child2)

research = Research(relationships)
# This will print:
# John is a parent of Jane
# John is a parent of Jack

# Solution

In [ ]:
from abc import ABC, abstractmethod
from enum import Enum
from typing import List, Tuple

# Enum for relationships
class Relationship(Enum):
    PARENT = 1
    CHILD = 2
    SIBLING = 3

# Person class
class Person:
    def __init__(self, name: str):
        self.name = name

# Relationships class (low-level module)
class Relationships:
    def __init__(self):
        self.relations: List[Tuple[Person, Relationship, Person]] = []

    def add_parent_and_child(self, parent: Person, child: Person):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.CHILD, parent))

# Abstract interface for relationship browsing
class RelationshipBrowser(ABC):
    @abstractmethod
    def find_all_children_of(self, name: str):
        pass

# Concrete implementation of RelationshipBrowser
class FixedRelationshipBrowser(RelationshipBrowser):
    def __init__(self, relationships: Relationships):
        self.relations = relationships.relations

    def find_all_children_of(self, name: str):
        for relation in self.relations:
            if relation[0].name == name and relation[1] == Relationship.PARENT:
                yield relation[2]

# Research class (high-level module)
class FixedResearch:
    def __init__(self, browser: RelationshipBrowser):
        for person in browser.find_all_children_of("John"):
            print(f"John is a parent of {person.name}")

# Example usage
parent = Person("John")
child1 = Person("Jane")
child2 = Person("Jack")

relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

browser = FixedRelationshipBrowser(relationships)
research = FixedResearch(browser)